In [4]:
import sys
sys.path.append('/home/vagrant/chainer/examples/text_classification')
import bottle
import chainer
import nets
import numpy
# scikit-learnで学習した分類器を保存するライブラリ
from sklern.externals import joblib

import dlclassifier
import mlclassifier
import ruleclassifier
import solrindexer as indexer
import sqlitedatastore as datastore
from annoutil import find_xs_in_y

# ルールによるテキスト分類の設定
rule = ruleclassifier.get_rule()

# 教師あり学習によるテキスト分類の設定
model_ml = joblib.load('../result/model.pkl')
vocab_ml = joblib.load('../result/vocab.pkl')

# ディープラーニングによるテキスト分類の設定
w = numpy.load('../result/w_dl.npy')
encoder = dlclassifier.Encoder(w)
model_dl = nets.TextClassifier(encoder, n_class=2)
chainer.serializers.load_npz('../result/model_dl.npz', model_dl)
with open('../result/vocab_dl.json') as f:
    vocab_dl = json.load(f)

# bottleのフレームワークはルーティングを1ファイル内ですべて完結できる
# 下記@bottle.route('/')内はホームルートで有ることを表す
@bottle.route('/')
def index_html():
    # ルート（http://localhost:8702）にアクセスした時に表示数rファイルをsearch.htmlと指定している。
    # rootにはフォルダまでを記述、ファイル名は第1引数に記述
    return bottle.static_file('search.html', root='../data/static')


@bottle.route('/file/<filename:path>')
def static(filename):
    # file(http://localhost:8702/file/)にアクセスした時に、data/staticフォルダの下のファイルを返す用にしている。
    return bottle.static_file(filename, root='../data/static')

# 下記はhttp://localhost:8702/getにアクセスした時に実行される
@bottle.get('/get')
def get():
    # リクエストの中からkeywordsと、classifierという変数を受け取る
    classifier_name = bottle.request.params.classifier
    # keywordsはsplit()で空白文字で区切ってリストにする
    keywords = bottle.request.params.keywords.split()
    # search関数を呼びresultsのリストに代入する
    # results = indexer.search(keywords=keywords_expanded)
    # search_annotaion関数を呼びresultsのリストに代入する
    results = indexer.search_annotation(
        fl_keyword_pairs=[('sentence_txt_ja', [keywords]), ('name_s', [['sentence']])],
        rows=1000)
    
    for r in results['response']['docs']:
        sent = datastore.get_annotation(r['doc_id_i'], 'sentence')[r['anno_id_i']]
        tokens = find_xs_in_y(datastore.get_annotation(r['doc_id_i'], 'token'), sent)
        
        # 分類がsvmだったら
        if classifier_name == 'ml':
            # 分類の実行
            features = mlclassifier.convert_into_features_using_vocab([(r['doc_id_i'], sent, tokens)], vocab_ml)
            # 正解の取得
            predicteds = mlclassifier.classify(features, model_ml)
        # LSTMだったら
        elif classifier_name == 'dl':
            features = dlclassifier.convert_into_features_using_vocab([(r['doc_id_i'], sent, tokens)], vocab_dl)
            predicteds = dlclassifier.classify(features, model_dl)
        # ruleベースだったら
        elif classifier_name == 'rule':
            features = ruleclassifier.convert_into_features_using_rules([(r['doc_id_i'], sent, tokens)], rule)
            predicteds = ruleclassifier.classify(features, rule)
            
        r['predicted'] = int(predicteds[0])
        print(r['predicted'], r['sentence_txt_ja'])
        
    # results変数をのjsonデータを返す
    return json.dumps(results, ensure_ascii=False)
        
            
    # keywordsの同義語を検索してkeywords_expandedに収納する
#     keywords_expanded = [[keyword] + [synonym['term']
#                                       for synonym in dbpediaknowledge.get_synonyms(keyword)]
#                                          for keyword in keywords]
    # inputのkeywordsが空を想定する
#     if keywords_expanded != []:
#         fl_keyword_pairs = [(name + '_txt_ja', keywords_expanded)]
#     else:
#         fl_keyword_pairs = [('name_s', [[name]])]
        
    # results変数をのjsonデータを返す
#     return json.dumps(results, ensure_ascii=False)

# このファイルがモジュールとして実行された場合にファイル名が__namae__に入る。モジュールではなくメインのプログラムとして実行された場合__main__が__namae__に入りtrueとなる
if __name__ == '__main__':
    
    datastore.connect()
# ポート番号を決めhttp://localhost:8702でアクセスできるようにしている
    bottle.run(host='0.0.0.0', port='8702')
    datastore.close()

ModuleNotFoundError: No module named 'sklern'